In [37]:
import pandas as pd
df = pd.read_csv("5year3.csv")
N=1898

In [38]:
df.head(5)

,日期,轮次,主队,比分,客队,主队指数,平指数,客队指数,分析,year,score1,score2,home,draw,away
0,09-12 19:30,1,富勒姆,0-3,阿森纳,4.72,3.83,1.67,欧/析,20-21,0,3,5.058723,4.104854,1.789845
1,09-12 22:00,1,水晶宫,1-0,南安普敦,2.91,3.19,2.54,欧/析,20-21,1,0,3.057895,3.352125,2.669090
2,09-13 00:30,1,利物浦,4-3,利兹联,1.26,5.93,10.18,欧/析,20-21,4,3,1.336251,6.288864,10.796060
3,09-13 03:00,1,西汉姆,0-2,纽卡斯尔,2.20,3.32,3.22,欧/析,20-21,0,2,2.345880,3.540147,3.433516
4,09-13 21:00,1,西布朗,0-3,莱切城,3.80,3.59,1.97,欧/析,20-21,0,3,3.987430,3.767072,2.067168


In [39]:
df['ans']=0

In [40]:
for i in range(N):
    if (df['score1'][i]>df['score2'][i]):
        df['ans'][i]=1
    if (df['score1'][i]==df['score2'][i]):
        df['ans'][i]=0
    if (df['score1'][i]<df['score2'][i]):
        df['ans'][i]=-1


<ipython-input-40-ea1a90b26067>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ans'][i]=-1
<ipython-input-40-ea1a90b26067>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ans'][i]=1
<ipython-input-40-ea1a90b26067>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ans'][i]=0


In [41]:
import numpy as np
from sklearn import metrics 
import math
x=np.array(df[['home','draw','away']])

In [42]:
# 生成随机数列 k-mean 用
RAM=np.random.permutation(N)
RAM

array([1401,  612,  124, ..., 1476,   67,   90])

In [43]:
def distance(a,b):
    if (np.linalg.norm(b)==0) or (np.linalg.norm(a)==0):
        return 0
    return np.dot(a,b)/np.linalg.norm(a)/np.linalg.norm(b)

In [45]:
import copy

In [55]:
def kmeans(X,k,limitation):
    # 返回中心坐标
    
    # initialize
    cent=X[0:k]
    newcent=X[0:k]
    clust=np.zeros(N)
    newclust=np.zeros(N)
    
    RAM=np.random.permutation(N)
    for i in range(k):
        cent[i]=X[RAM[i]]
    cnt=0
    
    # print(cent)
    while cnt<limitation:
        cnt+=1
        
        # renew cluster belonging
        for i in range(N):
            min_dis=0
            for j in range(k):
                if (distance(X[i],cent[j])-min_dis>0):
                    min_dis=distance(X[i],cent[j])
                    newclust[i]=j
        
        
        # renew center
        for j in range(k):
            newcent[j]=[0,0,0]
            c=0
            for i in range(N):
                if (newclust[i]==j):
                    newcent[j]=newcent[j]+X[i]
                    c+=1
            if (c!=0):
                newcent[j]=newcent[j]/c
        # print(newcent)
        
        # test ultimation
        flag=1
        # print("old clust")
        # print(clust[0:50])
        # print("new")
        # print(newclust[0:50])
        for i in range(N):
            if clust[i]!=newclust[i]:
                flag=0
                break
        if (flag==1):
            break
        cent=copy.deepcopy(newcent)
        clust=copy.deepcopy(newclust)
    return cent


In [56]:
def kmean_label(X,k,center):
    cnt=np.zeros(3*k).reshape(k,-1)
    ans=np.zeros(k)
    # 算出每个簇的选择（简单多数法）
    for i in range(N):
        min_dis=0
        ind=-1
        for j in range(k):
            if (distance(X[i],center[j])-min_dis>0):
                min_dis=distance(X[i],center[j])
                ind=j
        if(df['score1'][i]>df['score2'][i]):
            cnt[ind][0]+=1
        if(df['score1'][i]==df['score2'][i]):
            cnt[ind][1]+=1
        if(df['score1'][i]<df['score2'][i]):
            cnt[ind][2]+=1
        
    # ans 1 表示主队赢， 2表示平，3 客队赢
    for j in range(k):
        if(cnt[j][0]>=cnt[j][1])and(cnt[j][0]>=cnt[j][2]):
            ans[j]=1
        if(cnt[j][1]>cnt[j][0])and(cnt[j][1]>=cnt[j][2]):
            ans[j]=2
        if(cnt[j][2]>cnt[j][0])and(cnt[j][2]>cnt[j][1]):
            ans[j]=3
    return ans

In [57]:
def kmean_test(DF,center,label,k):
    cnt=0
    sum=0
    for i in range(DF.iloc[:,0].size):
        min_dis=0
        ind=-1
        X=np.array([ df['主队指数'][i] , df['平指数'][i] , df['客队指数'][i] ])
        for j in range(k):
            if (distance(center[j],X)-min_dis>0):
                min_dis=distance(X,center[j])
                ind=j
        if label[ind]==1 and df['score1'][i]>df['score2'][i]:
            cnt+=1
            sum+=df['主队指数'][i]
        if label[ind] and df['score1'][i]==df['score2'][i]:
            cnt+=1
            sum+=df['平指数'][i]
        if label[ind] and df['score1'][i]<df['score2'][i]:
            cnt+=1
            sum+=df['客队指数'][i]
    print(cnt,'      ',sum)
    return sum

In [58]:
def TEST():
    Max_money=0
    Max_k=0
    for K in range(13):
        testK=K+2
        print("k = ",testK)
        c1=kmeans(x,testK,500)
        lab1=kmean_label(x,testK,c1)
        tmp=kmean_test(df,c1,lab1,testK)
        if(tmp>Max_money):
            Max_money=tmp
            Max_k=testK
    return Max_k

In [59]:
TEST()

k =  2
1659        4453.5299999999925
k =  3
1796        4815.2699999999895
k =  4
1829        4929.58999999999
k =  5
1750        4680.989999999991
k =  6
1788        4790.139999999989
k =  7
1739        4651.41999999999
k =  8
1728        4622.599999999989
k =  9
1783        4774.82999999999
k =  10
1723        4609.919999999989
k =  11
1733        4635.6499999999905
k =  12
1728        4622.599999999989
k =  13
1747        4672.70999999999
k =  14
1744        4664.67999999999


4